# 125个经典技术指标择时分析
## 广发证券 2020年1月3日

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time
import os

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed
import indicatorJ

# 基础函数
import utilsJ

## 策略

### 策略主体

In [2]:
class gf125(bt.Strategy):
    
    params = (
        ('printlog', False),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep a reference to buyprice and sellprice in the data[0] dataseries
        self.buyprice = None
        self.sellprice = None
        
        # Add indicators
        self.ER = indicatorJ.ER(self.data)
        self.DPO = indicatorJ.DPO(self.data)
        self.POS = indicatorJ.POS(self.data)
        self.TII = indicatorJ.TII(self.data)
        self.ADTM = indicatorJ.ADTM(self.data)
        self.MADisplaced = indicatorJ.MADisplaced(self.data)
        self.T3 = indicatorJ.T3(self.data)
        self.VMA = indicatorJ.VMA(self.data)
        self.BIAS = indicatorJ.BIAS(self.data)
        self.TMA = indicatorJ.TMA(self.data)
        self.TYP = indicatorJ.TYP(self.data)

        #self.KDJ = indicatorJ.KDJ(self.data)
        self.MAAMT = indicatorJ.MAAMT(self.data)
        self.SROCVOL = indicatorJ.SROCVOL(self.data)

        self.ind_num = 13
        self.buy_votes = self.ER.lines.Buy + self.DPO.lines.Buy + self.POS.lines.Buy + self.TII.lines.Buy + \
                         self.ADTM.lines.Buy + self.MADisplaced.lines.Buy + self.T3.lines.Buy + self.VMA.lines.Buy + \
                         self.BIAS.lines.Buy + self.TMA.lines.Buy + self.TYP.lines.Buy + self.MAAMT.lines.Buy +  \
                         self.SROCVOL.lines.Buy
        self.sell_votes = self.ER.lines.Sell + self.DPO.lines.Sell + self.POS.lines.Sell + self.TII.lines.Sell + \
                          self.ADTM.lines.Sell + self.MADisplaced.lines.Sell + self.T3.lines.Sell + self.VMA.lines.Sell + \
                          self.BIAS.lines.Sell + self.TMA.lines.Sell + self.TYP.lines.Sell + self.MAAMT.lines.Sell + \
                          self.SROCVOL.lines.Sell
        


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
#        if self.order:
#            return
        if self.buy_votes[0] >= self.ind_num // 2 and self.getposition(self.data).size == 0:
            lots = np.floor(self.broker.getcash() / (100*self.data.close[0]))*100
            self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.data.close[0], lots, self.getposition(self.data).size))
            self.order = self.buy(size=lots)
        elif self.sell_votes[0] >= self.ind_num // 2 and self.getposition(self.data).size != 0:
            self.log('Sell CREATE (Close), Price: %.2f, Current Position: %i' % (self.data.close[0], self.getposition(self.data).size))
            self.order = self.close()

## 单股回测/测试

In [17]:
s_date = datetime.date(2019,12,31)
e_date = datetime.date(2022,10,25)
stock_code = '600588.SH'
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'


if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(gf125, printlog=True)


    # Create stock Data Feed
    if os.path.exists('.\\Data\\'+stock_code+'.csv'):
        df = pd.read_csv('.\\Data\\'+stock_code+'.csv', 
                    converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    else:
        df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)

    data = btfeed.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(200000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

Starting Portfolio Value: 200000.00
2021-11-12: BUY CREATE, Price: 32.84, Lots: 6000, Current Position: 0
2021-11-15: BUY EXECUTED, Price: 32.84, Lot:6000, Cash: 2958, Value: 197971
2022-01-28: Sell CREATE (Close), Price: 34.72, Current Position: 6000
2022-02-07: SELL EXECUTED, Price: 34.72, Lot:6000, Cash: 211278, Value: 211278
2022-02-07: OPERATION PROFIT, GROSS 11278.20, NET 11278.20
2022-10-17: BUY CREATE, Price: 19.35, Lots: 10900, Current Position: 0
2022-10-18: BUY EXECUTED, Price: 19.35, Lot:10900, Cash: 363, Value: 209970
Final Portfolio Value: 231770.20
Net Profit: 15.89%


# 沪深300回测/测试

In [21]:
s_date = datetime.date(2019,12,31)
e_date = datetime.date(2022,10,25)
stock_index = '000300.SH'
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'

if __name__ ==  '__main__':

    pro = ts.pro_api(token)
    #index_list = np.unique(pro.index_weight(index_code=stock_index, 
    #                          start_date=s_date.strftime('%Y%m%d'), 
    #                          end_date=e_date.strftime('%Y%m%d')).con_code).tolist()
    index_list = np.unique(pro.query('stock_basic', exchange='', list_status='L', fields='ts_code')).tolist()
    profit_dict = {}

    for stock_code in index_list:
        print(stock_code)
        try:
            # Create a cerebro entity
            cerebro = bt.Cerebro()

            # Add a strategy
            strats = cerebro.addstrategy(gf125, printlog=True) 


            # Create stock Data Feed
            if os.path.exists('.\\Data\\'+stock_code+'.csv'):
                df = pd.read_csv('.\\Data\\'+stock_code+'.csv', 
                            converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
            else:
                df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)

            data = btfeed.PandasData(dataname=df,fromdate=s_date,todate=e_date)

            # Add the index Data Feed to Cerebo
            cerebro.adddata(data)

            # Set cash inside the strategy
            cerebro.broker = bt.brokers.BackBroker(coc=True)   
            cerebro.broker.setcash(2000000)

            # Set commission
            #cerebro.broker.setcommission()

            # Print out the starting conditions
            start_value = cerebro.broker.getvalue()
            #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

            # Run over everything
            cerebro.run()

            # Print out the final result
            final_value = cerebro.broker.getvalue()
            #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
            print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))
            profit_dict[stock_code] = (final_value - start_value) / start_value * 100
        except:
            print('failed.')

000001.SZ
2020-12-29: BUY CREATE, Price: 18.68, Lots: 107000, Current Position: 0
2020-12-30: BUY EXECUTED, Price: 18.68, Lot:107000, Cash: 790, Value: 2003135
Net Profit: -42.98%
000002.SZ
2020-11-05: BUY CREATE, Price: 26.05, Lots: 76700, Current Position: 0
2020-11-06: BUY EXECUTED, Price: 26.05, Lot:76700, Cash: 2164, Value: 2014342
2021-03-31: Sell CREATE (Close), Price: 26.72, Current Position: 76700
2021-04-01: SELL EXECUTED, Price: 26.72, Lot:76700, Cash: 2051227, Value: 2051227
2021-04-01: OPERATION PROFIT, GROSS 51227.93, NET 51227.93
2021-12-03: BUY CREATE, Price: 18.15, Lots: 113000, Current Position: 0
2021-12-06: BUY EXECUTED, Price: 18.15, Lot:113000, Cash: 809, Value: 2074641
Net Profit: -12.10%
000004.SZ
2021-03-01: BUY CREATE, Price: 19.12, Lots: 104600, Current Position: 0
2021-03-02: BUY EXECUTED, Price: 19.12, Lot:104600, Cash: 48, Value: 1986401
2021-08-13: Sell CREATE (Close), Price: 21.12, Current Position: 104600
2021-08-16: SELL EXECUTED, Price: 21.12, Lot:104

In [ ]:
list(profit_dict.values())